In [19]:
## imports 
import pandas as pd
import numpy as np
import yaml
import warnings
warnings.filterwarnings("ignore", category=UserWarning)

# comment these out if you don't have plotnine--not essential here/only used once
import matplotlib.pyplot as plt
# import plotnine
# from plotnine import *

## way to connect to mysql 
## if you need to install
## uncomment this line:
#! pip install mysql-connector-python
import mysql.connector

## function to feed path name to load
## credentials
def load_creds(path: str):
    with open(path, 'r') as stream:
        try:
            creds = yaml.safe_load(stream)
        except yaml.YAMLError as exc:
            print(exc)
    return(creds)

pd.options.display.max_rows = 999
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

# Preliminary: define connection and read sample of data

In [20]:
## read in creds; change the path name if stored
## elsewhere
creds = load_creds("09_db_cred.yaml")

In [21]:
creds

{'practice_database': {'user': 'qss20',
  'password': 'xtPdDZHnxKMFSD5RyP',
  'port': 3306,
  'database': 'sentencing',
  'host': 'qss20-7570-db.c.dartmouth.edu'}}

In [22]:
cnx = mysql.connector.connect(user=creds['practice_database']['user'], 
                            password=creds['practice_database']['password'],
                            port=creds['practice_database']['port'],
                            database= creds['practice_database']['database'],
                            host = creds['practice_database']['host'])
cnx

# Activity 1

1. Create a new column -- `in_chicago` when pulling from the `caseinit` table that takes on the value of "YES" if INCIDENT_CITY = Chicago; "NO" otherwise (which represents incidents in Cook County suburbs outside the city limits);  and pull the table. Use `crosstabs` to confirm that this worked
2. Repeat step 1 but also filter out blank strings (`INCIDENT_CITY` == "")
3. Use `where` to row filter to initiations in Chicago and use group by to find the count of cases diverted and not diverted (`is_in_diversion`); pull the table with those counts
4. Modify the query in step 3 to find the proportion of cases in chicago diverted (hint you made need to use case when in a subquery)
5. Modify the query in step 4 to find the proportion of cases in chicago versus cases not in chicago sent to diversion 


In [23]:
# 1
compare_charge_city = """
SELECT *,
       CASE
         WHEN INCIDENT_CITY = "Chicago" THEN "YES"
         ELSE "NO"
       END AS in_chicago
FROM   caseinit
"""

read_sample_d = pd.read_sql_query(compare_charge_city, cnx)
read_sample_d

,CASE_ID,CASE_PARTICIPANT_ID,RECEIVED_DATE,OFFENSE_CATEGORY,PRIMARY_CHARGE_FLAG,CHARGE_ID,CHARGE_VERSION_ID,DISPOSITION_CHARGED_OFFENSE_TITLE,CHARGE_COUNT,DISPOSITION_DATE,...,LAW_ENFORCEMENT_AGENCY,LAW_ENFORCEMENT_UNIT,ARREST_DATE,FELONY_REVIEW_DATE,FELONY_REVIEW_RESULT,ARRAIGNMENT_DATE,UPDATED_OFFENSE_CATEGORY,is_in_diversion,in_chicago,in_chicago
0,198055620664,85937621020,08/15/1984 12:00:00 AM,PROMIS Conversion,False,1242195814523,155656315869,FIRST DEGREE MURDER,2,12/17/2014 12:00:00 AM,...,CHICAGO POLICE DEPT,,08/15/1984 12:00:00 AM,08/15/1984 12:00:00 AM,Charge(S) Approved,09/21/1984 12:00:00 AM,PROMIS Conversion,False,None,NO
1,198055620664,85937621020,08/15/1984 12:00:00 AM,PROMIS Conversion,False,1242198287388,131513547452,HOME INVASION,14,12/17/2014 12:00:00 AM,...,CHICAGO POLICE DEPT,,08/15/1984 12:00:00 AM,08/15/1984 12:00:00 AM,Charge(S) Approved,09/21/1984 12:00:00 AM,PROMIS Conversion,False,None,NO
2,198055620664,85937621020,08/15/1984 12:00:00 AM,PROMIS Conversion,False,1242351605056,176626576281,FIRST DEGREE MURDER,4,12/17/2014 12:00:00 AM,...,CHICAGO POLICE DEPT,,08/15/1984 12:00:00 AM,08/15/1984 12:00:00 AM,Charge(S) Approved,09/21/1984 12:00:00 AM,PROMIS Conversion,False,None,NO
3,198055620664,85937621020,08/15/1984 12:00:00 AM,PROMIS Conversion,False,1242352841488,176617824190,FIRST DEGREE MURDER,5,12/17/2014 12:00:00 AM,...,CHICAGO POLICE DEPT,,08/15/1984 12:00:00 AM,08/15/1984 12:00:00 AM,Charge(S) Approved,09/21/1984 12:00:00 AM,PROMIS Conversion,False,None,NO
4,198055620664,85937621020,08/15/1984 12:00:00 AM,PROMIS Conversion,False,1242356550787,131238606761,HOME INVASION,13,12/17/2014 12:00:00 AM,...,CHICAGO POLICE DEPT,,08/15/1984 12:00:00 AM,08/15/1984 12:00:00 AM,Charge(S) Approved,09/21/1984 12:00:00 AM,PROMIS Conversion,False,None,NO
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
272289,597320386308,452719800958,07/18/2022 12:00:00 AM,Retail Theft,True,11294724616133,1148300237684,RETAIL THEFT,1,09/02/2022 12:00:00 AM,...,CHICAGO PD,,07/18/2022 09:20:00 AM,07/18/2022 12:00:00 AM,Approved,09/02/2022 12:00:00 AM,Retail Theft,True,None,YES
272290,597328542979,452727325242,07/18/2022 12:00:00 AM,Aggravated Fleeing and Eluding,True,11295052270826,1148334245809,AGGRAVATED FLEEING OR ATTEMPT TO ELUDE A PEACE...,1,09/06/2022 12:00:00 AM,...,EVERGREEN PARK PD,,07/18/2022 11:11:00 PM,07/18/2022 12:00:00 AM,Approved,09/06/2022 12:00:00 AM,Aggravated Fleeing and Eluding,False,None,NO
272291,597341166398,452739650297,07/19/2022 12:00:00 AM,Criminal Damage to Property,True,11295731072434,1148405887926,CRIMINAL DAMAGE TO GOVERNMENT SUPPORTED PROPERTY,1,08/23/2022 12:00:00 AM,...,COOK COUNTY SHERIFF (IL0160000),,,08/02/2022 12:00:00 AM,Approved,08/17/2022 12:00:00 AM,Criminal Damage to Property,False,None,YES
272292,597341166398,452739650297,07/19/2022 12:00:00 AM,Criminal Damage to Property,True,11295731072434,1148405887926,CRIMINAL DAMAGE TO GOVERNMENT SUPPORTED PROPERTY,1,08/23/2022 12:00:00 AM,...,COOK COUNTY SHERIFF (IL0160000),,,08/02/2022 12:00:00 AM,Approved,08/17/2022 12:00:00 AM,Criminal Damage to Property,False,None,YES


In [24]:
# 2
compare_charge_city = """
SELECT *,
       CASE
         WHEN INCIDENT_CITY = "Chicago" THEN "YES"
         ELSE "NO"
       END AS in_chicago
FROM   caseinit
WHERE INCIDENT_CITY <> ""
limit 5
"""

read_sample_d = pd.read_sql_query(compare_charge_city, cnx)
read_sample_d

,CASE_ID,CASE_PARTICIPANT_ID,RECEIVED_DATE,OFFENSE_CATEGORY,PRIMARY_CHARGE_FLAG,CHARGE_ID,CHARGE_VERSION_ID,DISPOSITION_CHARGED_OFFENSE_TITLE,CHARGE_COUNT,DISPOSITION_DATE,...,LAW_ENFORCEMENT_AGENCY,LAW_ENFORCEMENT_UNIT,ARREST_DATE,FELONY_REVIEW_DATE,FELONY_REVIEW_RESULT,ARRAIGNMENT_DATE,UPDATED_OFFENSE_CATEGORY,is_in_diversion,in_chicago,in_chicago
0,288677981683,95130773152,08/13/2006 12:00:00 AM,PROMIS Conversion,True,8889814596849,969426374459,DUI LIC SUSP OR REVOKED,1,01/28/2015 12:00:00 AM,...,OAK PARK PD,,08/13/2006 02:53:00 AM,08/13/2006 12:00:00 AM,Charge(S) Approved,10/03/2012 12:00:00 AM,PROMIS Conversion,False,None,NO
1,289906337472,103165877915,06/18/2007 12:00:00 AM,Sex Crimes,False,6339673180732,590301665421,CRIMINAL SEXUAL ASSAULT,5,12/09/2013 12:00:00 AM,...,HARVEY POLICE DEPARTMENT,,08/30/2011 02:00:00 PM,08/30/2011 12:00:00 AM,Approved,10/06/2011 12:00:00 AM,Sex Crimes,False,None,NO
2,291517474172,103760942634,08/20/2008 12:00:00 AM,PROMIS Conversion,True,6363523969467,608781580816,AGGRAVATED FLEEING OR ATTEMPT TO ELUDE A PEACE...,1,01/18/2012 12:00:00 AM,...,MORTON GROVE PD,,08/20/2008 12:00:00 AM,08/20/2008 12:00:00 AM,Approved Arrest War,08/02/2010 12:00:00 AM,PROMIS Conversion,False,None,NO
3,291824708774,103847218017,09/06/2008 12:00:00 AM,PROMIS Conversion,True,6786219541928,701030496794,AGGRAVATED DRIVING UNDER THE INFLUENCE OF ALCOHOL,1,07/11/2012 12:00:00 AM,...,CHICAGO PD,,09/06/2008 12:45:00 AM,09/06/2008 12:00:00 AM,Charge(S) Approved,09/08/2011 12:00:00 AM,PROMIS Conversion,False,None,YES
4,292499187771,103592823349,02/26/2010 12:00:00 AM,PROMIS Conversion,True,8223789049105,866910505404,UNLAWFUL USE OF A WEAPON,1,08/22/2011 12:00:00 AM,...,CHICAGO POLICE DEPT,,02/25/2010 09:49:00 PM,02/26/2010 12:00:00 AM,Sf,05/28/2010 12:00:00 AM,PROMIS Conversion,False,None,YES


In [27]:
# 3
compare_charge_city = """
SELECT
    COUNT(*) as count_div,
    is_in_diversion

FROM   caseinit
WHERE INCIDENT_CITY = "Chicago"
GROUP BY is_in_diversion
"""
read_sample_d = pd.read_sql_query(compare_charge_city, cnx)
read_sample_d

,count_div,is_in_diversion
0,167171,False
1,6402,True


In [28]:
# 4

compare_charge_city = """
SELECT
                *
                CASE
                    WHEN is_in_diversion = "True" THEN 1
                    ELSE 0
                END  is_div_bool
            FROM caseinit
                

SyntaxError: incomplete input (1713353928.py, line 3)

In [28]:
# your code here 1

chicago_var = """
SELECT *,
       CASE
         WHEN INCIDENT_CITY = "Chicago" THEN 'YES'
         ELSE 'NO'
       END AS in_chicago
FROM   caseinit 
where INCIDENT_CITY != ""
limit 5
"""
read_sample_d = pd.read_sql_query(chicago_var, cnx)
read_sample_d

,CASE_ID,CASE_PARTICIPANT_ID,RECEIVED_DATE,OFFENSE_CATEGORY,PRIMARY_CHARGE_FLAG,CHARGE_ID,CHARGE_VERSION_ID,DISPOSITION_CHARGED_OFFENSE_TITLE,CHARGE_COUNT,DISPOSITION_DATE,...,LAW_ENFORCEMENT_AGENCY,LAW_ENFORCEMENT_UNIT,ARREST_DATE,FELONY_REVIEW_DATE,FELONY_REVIEW_RESULT,ARRAIGNMENT_DATE,UPDATED_OFFENSE_CATEGORY,is_in_diversion,in_chicago,in_chicago
0,288677981683,95130773152,08/13/2006 12:00:00 AM,PROMIS Conversion,True,8889814596849,969426374459,DUI LIC SUSP OR REVOKED,1,01/28/2015 12:00:00 AM,...,OAK PARK PD,,08/13/2006 02:53:00 AM,08/13/2006 12:00:00 AM,Charge(S) Approved,10/03/2012 12:00:00 AM,PROMIS Conversion,False,None,NO
1,289906337472,103165877915,06/18/2007 12:00:00 AM,Sex Crimes,False,6339673180732,590301665421,CRIMINAL SEXUAL ASSAULT,5,12/09/2013 12:00:00 AM,...,HARVEY POLICE DEPARTMENT,,08/30/2011 02:00:00 PM,08/30/2011 12:00:00 AM,Approved,10/06/2011 12:00:00 AM,Sex Crimes,False,None,NO
2,291517474172,103760942634,08/20/2008 12:00:00 AM,PROMIS Conversion,True,6363523969467,608781580816,AGGRAVATED FLEEING OR ATTEMPT TO ELUDE A PEACE...,1,01/18/2012 12:00:00 AM,...,MORTON GROVE PD,,08/20/2008 12:00:00 AM,08/20/2008 12:00:00 AM,Approved Arrest War,08/02/2010 12:00:00 AM,PROMIS Conversion,False,None,NO
3,291824708774,103847218017,09/06/2008 12:00:00 AM,PROMIS Conversion,True,6786219541928,701030496794,AGGRAVATED DRIVING UNDER THE INFLUENCE OF ALCOHOL,1,07/11/2012 12:00:00 AM,...,CHICAGO PD,,09/06/2008 12:45:00 AM,09/06/2008 12:00:00 AM,Charge(S) Approved,09/08/2011 12:00:00 AM,PROMIS Conversion,False,None,YES
4,292499187771,103592823349,02/26/2010 12:00:00 AM,PROMIS Conversion,True,8223789049105,866910505404,UNLAWFUL USE OF A WEAPON,1,08/22/2011 12:00:00 AM,...,CHICAGO POLICE DEPT,,02/25/2010 09:49:00 PM,02/26/2010 12:00:00 AM,Sf,05/28/2010 12:00:00 AM,PROMIS Conversion,False,None,YES


In [14]:
# your code here 2


chicago_var = """
SELECT *,
       CASE
           WHEN INCIDENT_CITY = 'Chicago' THEN 'YES'
           ELSE 'NO'
       END AS in_chicago
FROM caseinit
WHERE INCIDENT_CITY <> '';
limit 5
"""
read_sample_d = pd.read_sql_query(chicago_var, cnx)
read_sample_d

DatabaseError: Execution failed on sql: 
SELECT *,
       CASE
           WHEN INCIDENT_CITY = 'Chicago' THEN 'YES'
           ELSE 'NO'
       END AS in_chicago
FROM caseinit
WHERE INCIDENT_CITY <> '';
limit 5

Use multi=True when executing multiple statements
unable to rollback

In [ ]:
# your code here 3

chicago_var = """
SELECT updated_offense_category,
       Count(*) AS count_offense
FROM   caseinit
WHERE  race IN ( "black", "white" )
GROUP  BY updated_offense_category
ORDER  BY count_offense DESC
LIMIT  5 
"""


t5_d = pd.read_sql_query(t5, cnx)
t5_d

In [ ]:
# your code here 4

In [ ]:
# your code here 5

# Activity 2 

1. Use the following crosswalk and the `CASE` variable in the `divert` table to create a new variable `DIVERSION_PROGRAM_TEXT` that spells out the diversion programs
    - DC: Drug Court

    - DDPP: Drug Deferred Prosecution

    - DS: Drug School

    - RJCC: Restorative Justice

    - MHC: Mental Health Court

    - VC: Veteran Court

2. Build on the query from step 1 to filter to Narcotics as the `UPDATED_OFFENSE_CATEGORY` and Black or White defendants (based on race in the diversions table) (hint: you'll need to join with the caseinit table based on case_id and case_participant_id, you can do a inner join to keep only those diverted). Select the case_id, case_participant_id, case, race, and diversion_program_text columns

In [ ]:
# your code here 1

In [ ]:
# your code here 2